In [ ]:
# 앞서 배운 메모리 클래스 중 하나를 사용하는 메모리로 LCEL 체인을 구현합니다.
# 이 체인은 영화 제목을 가져와 영화를 나타내는 세 개의 이모티콘으로 응답해야 합니다. (예: "탑건" -> "🛩️👨‍✈️🔥". "대부" -> "👨‍👨‍👦🔫🍝").
# 항상 세 개의 이모티콘으로 답장하도록 FewShotPromptTemplate 또는 FewShotChatMessagePromptTemplate을 사용하여 체인에 예시를 제공하세요.
# 메모리가 작동하는지 확인하려면 체인에 두 개의 영화에 대해 질문한 다음 다른 셀에서 체인에 먼저 질문한 영화가 무엇인지 알려달라고 요청하세요.

from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder, FewShotChatMessagePromptTemplate

llm = ChatOpenAI(
    temperature=0.1,
    model="gpt-3.5-turbo",
    
)

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)

examples = [
    {
        "movie_name": "탑건",
        "answer": "🛩️✈️🔥",
    },
    {
        "movie_name": "대부",
        "answer": "👨‍👨‍👦🔫🍝",
    },
    {
        "movie_name": "인셉션",
        "answer": "🧠💤🌌",
    },
    {
        "movie_name": "터미네이터",
        "answer": "🤖🔫⏳",
    },
    {
        "movie_name": "스타워즈",
        "answer": "⭐🌌👽",
    },
]

example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What emojis represent the movie {movie_name}?"),
        ("ai", "{answer}"),
    ]
)


few_shot_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)


final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are movie expert. but you can answer with three emoji when user ask the movie name"),
        few_shot_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)


def load_memory(_):
    return memory.load_memory_variables({})["history"]


chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm


def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)


# 체인 실행
invoke_chain("태극기를휘날리며")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter,CharacterTextSplitter

splitter = RecursiveCharacterTextSplitter()

splitter_2 = CharacterTextSplitter(separator="\n",chunk_size=600,chunk_overlap=100,)

loader = UnstructuredFileLoader("./files/chapter_one.docx")

loader.load_and_split(text_splitter=splitter)